In [ ]:
# pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1

In [28]:
import json
with open("./2WikiMultihopQA/train.json") as f:
    wiki = json.load(f)

In [29]:
for w in wiki:
    w_context = ""
    sup_context = ""
    supporting_facts = {fact[0]: fact[1] for fact in w["supporting_facts"]}
    l = w["context"]
    prev = 0
    sent_num = []
    for i in range(len(l)-1):
        if l[i][0] in supporting_facts:
            idx = supporting_facts[l[i][0]]
            for id, text in enumerate(l[i][1]):
                if id == idx:
                    sup_context += f"{text}({id+prev+1}) "
                    sent_num.append(id+prev+1)
        w_context += " ".join([f"{text}({id+prev+1})" for id, text in enumerate(l[i][1])])
        prev+=len(l[i][1])
    w["w_context"] = w_context
    w["sup_context"] = sup_context
    w["sent_num"] = sent_num



In [30]:
wiki[0]['sup_context'], wiki[0]['sent_num']

('Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).(1) Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.(3) Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart Rosenberg.(7) Jean-Daniel Pollet (1936–2004) was a French film director and screenwriter who was most active in the 1960s and 1970s.(14) ',
 [1, 3, 7, 14])

In [31]:
# Preprocessing function
def preprocess(example):
    input_text = f"question: {example['question']}, context: {example['w_context']}"
    output_text = f"answer: {example['answer']}, reference: {example['sent_num']}"
    return {"input_text": input_text, "output_text": output_text}

dataset = list(map(preprocess, wiki))

In [32]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import os
# os.environ["WANDB_DISABLED"] = "true"

dataset = Dataset.from_list(dataset)

In [ ]:

# device = "mps" if torch.backends.mps.is_available() else "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load model & tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained('gpt2')
# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)

# Tokenization function
def tokenize_function(example):
    return tokenizer(example["input_text"], text_target=example["output_text"], padding="max_length", truncation=True)

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 167454/167454 [01:34<00:00, 1766.59 examples/s]


In [34]:
dataset = tokenized_datasets.remove_columns(['input_text', 'output_text'])
dataset = dataset.train_test_split(test_size=0.1)

In [43]:
# model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)
# Define training arguments
training_args = TrainingArguments(
    output_dir = "./Shared-Task-2025",
    bf16=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test']
)

# Train the model
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

NameError: name 'fp16' is not defined